# **Load Dataset**

In [1]:
# library dasar
import os
import pandas as pd
from PIL import Image

# library PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [16]:
class EmotionDataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.df = pd.read_csv(csv_file)
        self.transform = transform

        assert "image" in self.df.columns
        assert "label" in self.df.columns

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["image"]
        
        # label asli (1–7)
        label_raw = int(self.df.iloc[idx]["label"])
        
        # convert ke 0–6 untuk CrossEntropyLoss
        label = label_raw - 1

        # path gambar (tetap pakai folder label 1–7)
        img_path = os.path.join(self.root_dir, str(label_raw), img_name)

        # load gambar
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [6]:
def get_dataloaders(batch_size=32):
    # transformasi untuk semua gambar
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # ubah ukuran gambar
        transforms.ToTensor(),          # ubah ke tensor
    ])

    # dataset untuk train
    train_dataset = EmotionDataset(
        root_dir="train",
        csv_file="train_labels.csv",
        transform=transform
    )

    # dataset untuk test
    test_dataset = EmotionDataset(
        root_dir="test",
        csv_file="test_labels.csv",
        transform=transform
    )

    # loader untuk mini-batch training
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # loader untuk test (tidak di-shuffle)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [7]:
# load data
train_loader, test_loader = get_dataloaders(batch_size=32)

# ambil 1 batch
images, labels = next(iter(train_loader))

print("Ukuran batch gambar:", images.shape)  # harus (32, 3, 224, 224)
print("Label batch:", labels[:10])           # tampilkan 10 label pertama


Ukuran batch gambar: torch.Size([32, 3, 224, 224])
Label batch: tensor([4, 1, 4, 7, 7, 4, 7, 4, 5, 1])


# **Modeling**

In [8]:
# import
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18

In [9]:
# Model ResNet18
def build_model(num_classes=7):
    # load ResNet18 pretrained
    model = resnet18(weights='IMAGENET1K_V1')
    
    # ganti fully connected layer terakhir
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    return model

In [11]:
# Inisialisasi Model + Loss + Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = build_model(num_classes=7).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Asus/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [02:42<00:00, 289kB/s] 


In [17]:
# trainning loop 
def train(model, train_loader, test_loader, epochs=10):
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            # forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # hitung akurasi training
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total

        # evaluasi setiap epoch
        test_acc = evaluate(model, test_loader)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Loss: {total_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | "
              f"Test Acc: {test_acc:.2f}%")

    return model


In [18]:
# Function Evaluasi
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total


In [19]:
# Training
train_loader, test_loader = get_dataloaders(batch_size=32)

model = train(model, train_loader, test_loader, epochs=10)


KeyboardInterrupt: 

In [ ]:
# Save Model
torch.save(model.state_dict(), "emotion_resnet18.pth")
print("Model saved!")
